In [1]:
from pathlib import Path
import sys

BASE_DIR = Path.cwd() if Path.cwd().name != "notebooks" else Path.cwd().parent
sys.path.append(str(BASE_DIR))

from src.cleaning import clean_all_files, export_processed
from src.validation import validate_dataset
from src.metrics import kpis, by_site_month, category_split, peak_days, sqri


In [2]:
df_clean = clean_all_files(BASE_DIR / "data" / "raw")
export_processed(df_clean, BASE_DIR / "data" / "processed" / "refunds_clean.csv")


c:\hotel-refund-intelligence\src\cleaning.py:93: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(s, errors="coerce", dayfirst=True)
c:\hotel-refund-intelligence\src\cleaning.py:93: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(s, errors="coerce", dayfirst=True)
c:\hotel-refund-intelligence\src\cleaning.py:93: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(s, errors="coerce", dayfirst=True)
c:\hotel-refund-intelligence\src\cleaning.py:93: UserWarning: Could not infer format, so each element will be parsed individ

In [1]:
import os
from pathlib import Path

print("cwd:", os.getcwd())
print("cwd exists:", Path(os.getcwd()).exists())
print("files in cwd:", list(Path(os.getcwd()).iterdir())[:10])


cwd: c:\Projects\hotel-refund-intelligence\notebooks
cwd exists: True
files in cwd: [WindowsPath('c:/Projects/hotel-refund-intelligence/notebooks/01_ingestion_profiling.ipynb'), WindowsPath('c:/Projects/hotel-refund-intelligence/notebooks/02_cleaning_validation.ipynb'), WindowsPath('c:/Projects/hotel-refund-intelligence/notebooks/03_analysis_storytelling.ipynb')]


In [2]:
from pathlib import Path
import os

# project root = parent of notebooks folder
BASE_DIR = Path.cwd()
if BASE_DIR.name == "notebooks":
    BASE_DIR = BASE_DIR.parent

# If you run notebook from elsewhere, fallback: search upward for README.md
if not (BASE_DIR / "README.md").exists():
    for p in Path.cwd().parents:
        if (p / "README.md").exists():
            BASE_DIR = p
            break

DATA_RAW = BASE_DIR / "data" / "raw"
DATA_INTERIM = BASE_DIR / "data" / "interim"
DATA_PROCESSED = BASE_DIR / "data" / "processed"
FIG_DIR = BASE_DIR / "reports" / "figures"

print("✅ BASE_DIR set to:", BASE_DIR)
print("✅ DATA_RAW:", DATA_RAW)


✅ BASE_DIR set to: c:\Projects\hotel-refund-intelligence
✅ DATA_RAW: c:\Projects\hotel-refund-intelligence\data\raw


In [3]:
import sys
print(sys.executable)


c:\Projects\hotel-refund-intelligence\.venv\Scripts\python.exe


# 01 — Ingestion & Profiling
Project: Hotel
 Refund Intelligence (Brighton vs Newhaven)

Goal:
- Load 6 raw CSVs (Brighton/Newheaven x Nov/Dec/Jan)
- Add metadata columns: site, file_month (derived from filename)
- Run profiling to understand schema, missingness, refund markers, and date fields
- Save a combined interim dataset (no cleaning applied yet)


In [15]:
from pathlib import Path
import re
import pandas as pd

RAW_DIR = Path("../data/raw")  # notebooks/ is one level down from project root
RAW_FILES = sorted(RAW_DIR.glob("*.csv"))

RAW_FILES


[WindowsPath('../data/raw/Brighton_December_refund.csv'),
 WindowsPath('../data/raw/Brighton_January_refund.csv'),
 WindowsPath('../data/raw/Brighton_November_refund.csv'),
 WindowsPath('../data/raw/Newheaven_December_refund.csv'),
 WindowsPath('../data/raw/Newheaven_January_refund.csv'),
 WindowsPath('../data/raw/Newheaven_November_refund.csv')]

In [16]:
import os
os.getcwd()


'c:\\Projects\\hotel-refund-intelligence\\notebooks'

In [17]:
MONTH_MAP = {
    "november": "Nov-2025",
    "december": "Dec-2025",
    "january":  "Jan-2026",
}

def parse_metadata_from_filename(filename: str) -> tuple[str, str]:
    """
    Expected patterns like:
    Brighton_November_refund.csv
    Newheaven_January_refund.csv
    """
    base = Path(filename).name
    parts = base.split("_")
    if len(parts) < 3:
        raise ValueError(f"Unexpected filename format: {base}")

    site_raw = parts[0].strip()
    month_raw = parts[1].strip().lower()

    # Standardize site name to match project spec (Brighton/Newhaven)
    # NOTE: your file uses 'Newheaven' spelling; we map it to 'Newhaven' as standard.
    if site_raw.lower() == "brighton":
        site = "Brighton"
    elif site_raw.lower() in ["newheaven", "newhaven"]:
        site = "Newhaven"
    else:
        site = site_raw  # fallback

    if month_raw not in MONTH_MAP:
        raise ValueError(f"Month not recognized in filename: {base}")

    file_month = MONTH_MAP[month_raw]
    return site, file_month

# quick test
for f in RAW_FILES:
    print(f.name, "->", parse_metadata_from_filename(f.name))


Brighton_December_refund.csv -> ('Brighton', 'Dec-2025')
Brighton_January_refund.csv -> ('Brighton', 'Jan-2026')
Brighton_November_refund.csv -> ('Brighton', 'Nov-2025')
Newheaven_December_refund.csv -> ('Newhaven', 'Dec-2025')
Newheaven_January_refund.csv -> ('Newhaven', 'Jan-2026')
Newheaven_November_refund.csv -> ('Newhaven', 'Nov-2025')


In [18]:
dfs = []

for f in RAW_FILES:
    site, file_month = parse_metadata_from_filename(f.name)
    df = pd.read_csv(f)
    df["site"] = site
    df["file_month"] = file_month
    df["source_file"] = f.name  # extra lineage (helpful for audit)
    dfs.append(df)

combined = pd.concat(dfs, ignore_index=True)

combined.shape, combined.columns[:15]


((133, 56),
 Index(['IS_INTERNAL_YN', 'INTERNAL_DEBIT', 'INTERNAL_CREDIT', 'FIRST',
        'FIRST_DEBIT', 'FIRST_CREDIT', 'SECOND', 'SECOND_DEBIT',
        'SECOND_CREDIT', 'THIRD', 'THIRD_DEBIT', 'THIRD_CREDIT', 'EXP_DATE',
        'RECEIPT_NO', 'GUEST_FULL_NAME'],
       dtype='str'))

In [19]:
# Basic schema overview
print("Rows:", len(combined))
print("Cols:", len(combined.columns))

# Columns that look like Unnamed:*
unnamed_cols = [c for c in combined.columns if str(c).lower().startswith("unnamed:")]
print("Unnamed columns:", unnamed_cols)

# Missingness summary (top 25 most-missing columns)
missing = combined.isna().mean().sort_values(ascending=False)
missing.head(25)


Rows: 133
Cols: 56
Unnamed columns: ['Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52']


SECOND                 1.000000
THIRD                  1.000000
EXP_DATE               1.000000
Unnamed: 51            0.977444
Unnamed: 52            0.977444
Unnamed: 49            0.962406
Unnamed: 50            0.962406
Unnamed: 48            0.932331
Unnamed: 47            0.924812
RECEIPT_NO             0.917293
MARKET_CODE            0.902256
Unnamed: 45            0.894737
Unnamed: 46            0.879699
USER_NAME              0.864662
CC_CODE                0.849624
FIRST                  0.218045
CURRENCY1              0.172932
DEPOSIT_DEBIT          0.165414
Unnamed: 43            0.150376
INSERT_DATE            0.142857
TARGET_RESORT          0.142857
Unnamed: 44            0.142857
CASHIER_NAME           0.135338
TRX_DESC               0.135338
PRINT_CASHIER_DEBIT    0.135338
dtype: float64

In [20]:
# Ensure BUSINESS_FORMAT_DATE exists (we'll validate formally in Notebook 02)
col = "BUSINESS_FORMAT_DATE"
if col not in combined.columns:
    print(f"WARNING: {col} not found. Columns available:", list(combined.columns))
else:
    refund_mask = combined[col].astype(str).str.contains("refund", case=False, na=False)
    print("Refund rows (by indicator):", refund_mask.sum())
    print("Refund %:", round(refund_mask.mean() * 100, 2), "%")
    combined.loc[refund_mask, col].value_counts().head(10)


Refund rows (by indicator): 65
Refund %: 48.87 %


In [21]:
for candidate in ["BUSINESS_DATE", "BUSINESS_TIME", "BUSINESS_FORMAT_DATE"]:
    if candidate in combined.columns:
        sample = combined[candidate].dropna().astype(str).head(5).tolist()
        print(candidate, "sample:", sample)
    else:
        print(candidate, "MISSING")


BUSINESS_DATE sample: ['19:10', '26/12/2025', '26/12/2025', '02-Dec-25', '27/12/2025']
BUSINESS_TIME sample: ['01/12/2025', 'OTH', 'OTH', '13:44', 'OTH']
BUSINESS_FORMAT_DATE sample: ['OTH', 'Accomm. Refund/Correction', 'Accomm. Refund/Correction', '02/12/2025', 'F&B Refund/Correction']


In [22]:
combined_for_export = combined.copy()

# Convert all object columns to pandas string dtype to avoid pyarrow mixed-type errors
obj_cols = combined_for_export.select_dtypes(include=["object"]).columns
combined_for_export[obj_cols] = combined_for_export[obj_cols].astype("string")

print("Converted object columns to string:", len(obj_cols))
print("Example:", list(obj_cols)[:10])



Converted object columns to string: 43
Example: ['IS_INTERNAL_YN', 'INTERNAL_DEBIT', 'INTERNAL_CREDIT', 'RECEIPT_NO', 'GUEST_FULL_NAME', 'TARGET_RESORT', 'TRX_DESC', 'MARKET_CODE', 'BUSINESS_FORMAT_DATE', 'BUSINESS_TIME']


C:\Users\DALVI\AppData\Local\Temp\ipykernel_21492\377403670.py:4: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  obj_cols = combined_for_export.select_dtypes(include=["object"]).columns


In [23]:
import pandas as pd

# show object columns and their missingness
obj_cols = combined.select_dtypes(include=["object"]).columns.tolist()
print("Object columns:", len(obj_cols))

# show top 20 object cols with most missing values
combined[obj_cols].isna().mean().sort_values(ascending=False).head(20)
col = "CASHIER_CREDIT"
print("dtype:", combined[col].dtype)
print("python types seen (top):")
print(combined[col].map(lambda x: type(x).__name__).value_counts().head(10))
combined[col].head(10)


Object columns: 43
dtype: object
python types seen (top):
CASHIER_CREDIT
str      71
float    62
Name: count, dtype: int64


C:\Users\DALVI\AppData\Local\Temp\ipykernel_21492\1210985226.py:4: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  obj_cols = combined.select_dtypes(include=["object"]).columns.tolist()


0             0
1    1090095721
2    1090095723
3             0
4    1091293970
5    1056708191
6    1075882805
7    1052649417
8    1052699998
9    1056936990
Name: CASHIER_CREDIT, dtype: object

In [24]:
combined_for_export = combined.copy()

# Convert every object column to pandas "string" dtype
# This handles mixed floats/strings cleanly.
for c in combined_for_export.columns:
    if combined_for_export[c].dtype == "object":
        combined_for_export[c] = combined_for_export[c].astype("string")

combined_for_export.dtypes.head(30)


IS_INTERNAL_YN                str
INTERNAL_DEBIT                str
INTERNAL_CREDIT               str
FIRST                     float64
FIRST_DEBIT               float64
FIRST_CREDIT              float64
SECOND                    float64
SECOND_DEBIT              float64
SECOND_CREDIT             float64
THIRD                     float64
THIRD_DEBIT               float64
THIRD_CREDIT              float64
EXP_DATE                  float64
RECEIPT_NO                 string
GUEST_FULL_NAME               str
TARGET_RESORT                 str
TRX_DESC                      str
MARKET_CODE                string
BUSINESS_FORMAT_DATE          str
BUSINESS_TIME                 str
BUSINESS_DATE                 str
REFERENCE                     str
TRX_NO                        str
CASHIER_DEBIT                 str
CASHIER_CREDIT             string
ROOM                       string
CREDIT_CARD_SUPPLEMENT     string
CURRENCY1                  string
TRX_CODE                      str
CASHIER_ID    

In [25]:
from pathlib import Path

INTERIM_DIR = Path("../data/interim") if Path("../data").exists() else Path("data/interim")
INTERIM_DIR.mkdir(parents=True, exist_ok=True)

out_path = INTERIM_DIR / "combined_raw_with_metadata.parquet"
combined_for_export.to_parquet(out_path, index=False)

out_path, out_path.exists()


(WindowsPath('../data/interim/combined_raw_with_metadata.parquet'), True)